In [1]:
import dask.dataframe as dd

In [2]:
%%time
df = dd.read_csv('nyctaxi/raw/2016_Yellow_Taxi_Trip_Data.csv')

CPU times: user 36.4 ms, sys: 8.87 ms, total: 45.2 ms
Wall time: 73.1 ms


In [7]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
npartitions=271,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,float64,float64,int64,object,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:46267 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [5]:
from tqdm import tqdm
for year in tqdm([2016, 2017, 2018]):
    fn = f"nyctaxi/raw/{year}_Yellow_Taxi_Trip_Data.csv"
    fldr_out = f"nyctaxi/part_csv_stage1/year={year}/part_*.csv"
    df = dd.read_csv(fn)
    out = df.to_csv(fldr_out, index=False)

100%|██████████| 3/3 [51:10<00:00, 1023.54s/it]


In [3]:
from tqdm import tqdm
for year in tqdm([2016, 2017, 2018]):
    fn = f"nyctaxi/raw/{year}_Yellow_Taxi_Trip_Data.csv"
    fldr_out = f"nyctaxi/part_parquet_stage1/year={year}/"
    df = dd.read_csv(fn)
    out = df.to_parquet(fldr_out, write_index=False, write_metadata_file=False)

100%|██████████| 3/3 [14:19<00:00, 286.44s/it]


In [1]:
import dask.dataframe as dd

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:44383 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.50 GB


In [8]:
df = dd.read_csv('nyctaxi/part_csv_stage1/*/*')

In [12]:
len(df.columns)

21

In [8]:
df = dd.read_parquet('nyctaxi/part_parquet_stage1/')

In [5]:
len(df.columns)

22

In [10]:
df["passenger_count"].min().compute()

0

In [11]:
df["passenger_count"].max().compute()

192

In [6]:
df = dd.read_parquet('nyctaxi/part_parquet_stage1/*/*')

In [7]:
len(df.columns)

21

In [22]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
npartitions=605,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,float64,float64,int64,object,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [18]:
df = dd.read_csv("nyctaxi/raw/*")

In [19]:
len(df.columns)

21

In [28]:
def extract_dtypes(df, name): 
    dtypes = df.dtypes.astype(str).to_frame(name=name) 
    return dtypes 

In [31]:
df_types = [] 
for year in [2016, 2017, 2018]:
    fn = f"nyctaxi/raw/{year}_Yellow_Taxi_Trip_Data.csv"
    df = dd.read_csv(fn)
    df_types.append(extract_dtypes(df, year))
df_types = pd.concat(df_types, axis=1)

In [34]:
df_types

,2016,2017,2018
VendorID,int64,int64,int64
tpep_pickup_datetime,object,object,object
tpep_dropoff_datetime,object,object,object
passenger_count,int64,int64,int64
trip_distance,float64,float64,float64
pickup_longitude,float64,NaN,NaN
pickup_latitude,float64,NaN,NaN
RatecodeID,int64,int64,int64
store_and_fwd_flag,object,object,object
dropoff_longitude,float64,NaN,NaN


In [41]:
mask = df_types.isnull().any(1)
df_types[mask]

,2016,2017,2018
pickup_longitude,float64,NaN,NaN
pickup_latitude,float64,NaN,NaN
dropoff_longitude,float64,NaN,NaN
dropoff_latitude,float64,NaN,NaN
